## Preparing for Data Scraping

Public Data Source: https://www.seoul.go.kr/coronaV/coronaStatus.do

In [546]:
import pandas as pd
import numpy as np

In [547]:
import requests

In [548]:
url = "https://news.seoul.go.kr/api/27/getCorona19Status/get_status_ajax.php?draw=1"
#url = f"{url}&columns%5B0%5D%5Bdata%5D=0&columns%5B0%5D%5Bname%5D=&columns%5B0%5D%5Bsearchable%5D=true&columns%5B0%5D%5Borderable%5D=true&columns%5B0%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B0%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B1%5D%5Bdata%5D=1&columns%5B1%5D%5Bname%5D=&columns%5B1%5D%5Bsearchable%5D=true&columns%5B1%5D%5Borderable%5D=true&columns%5B1%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B1%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B2%5D%5Bdata%5D=2&columns%5B2%5D%5Bname%5D=&columns%5B2%5D%5Bsearchable%5D=true&columns%5B2%5D%5Borderable%5D=true&columns%5B2%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B2%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B3%5D%5Bdata%5D=3&columns%5B3%5D%5Bname%5D=&columns%5B3%5D%5Bsearchable%5D=true&columns%5B3%5D%5Borderable%5D=true&columns%5B3%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B3%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B4%5D%5Bdata%5D=4&columns%5B4%5D%5Bname%5D=&columns%5B4%5D%5Bsearchable%5D=true&columns%5B4%5D%5Borderable%5D=true&columns%5B4%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B4%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B5%5D%5Bdata%5D=5&columns%5B5%5D%5Bname%5D=&columns%5B5%5D%5Bsearchable%5D=true&columns%5B5%5D%5Borderable%5D=true&columns%5B5%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B5%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B6%5D%5Bdata%5D=6&columns%5B6%5D%5Bname%5D=&columns%5B6%5D%5Bsearchable%5D=true&columns%5B6%5D%5Borderable%5D=true&columns%5B6%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B6%5D%5Bsearch%5D%5Bregex%5D=false&order%5B0%5D%5Bcolumn%5D=0&order%5B0%5D%5Bdir%5D=desc"
url = f"{url}&start=0&length=100"
#"&search%5Bvalue%5D=&search%5Bregex%5D=true&_=1628930508535"
url

'https://news.seoul.go.kr/api/27/getCorona19Status/get_status_ajax.php?draw=1&start=0&length=100'

In [549]:
response = requests.get(url)
data_json = response.json()
# pd.DataFrame(data_json['data'])


In [550]:
records_total = data_json['recordsTotal']
records_total

41720

In [551]:
end_page = round(records_total/ 100) + 1
end_page                                        # used in get_multi_page_list function

418

In [552]:
data = data_json["data"]

In [553]:
pd.DataFrame(data)

,0,1,2,3,4,5,6
0,<p class='corona19_no'>71720</p>,225438,2021-08-15,기타,-,감염경로 조사중,<b class=''>-</b>
1,<p class='corona19_no'>71719</p>,225111,2021-08-15,성북구,-,기타 확진자 접촉,<b class=''>-</b>
2,<p class='corona19_no'>71718</p>,225046,2021-08-15,서대문구,-,타시도 확진자 접촉,<b class=''>-</b>
3,<p class='corona19_no'>71717</p>,225288,2021-08-15,강남구,-,감염경로 조사중,<b class=''>-</b>
4,<p class='corona19_no'>71716</p>,224003,2021-08-15,동대문구,-,감염경로 조사중,<b class=''>-</b>
...,...,...,...,...,...,...,...
95,<p class='corona19_no'>71625</p>,224877,2021-08-15,성동구,-,기타 확진자 접촉,<b class=''>-</b>
96,<p class='corona19_no'>71624</p>,224582,2021-08-15,타시도,-,감염경로 조사중,<b class=''>-</b>
97,<p class='corona19_no'>71623</p>,224870,2021-08-15,마포구,-,감염경로 조사중,<b class=''>-</b>
98,<p class='corona19_no'>71622</p>,224926,2021-08-15,타시도,-,송파구 소재 물류센터 관련('21.8),<b class=''>-</b>


## Function for scraping one pages

In [554]:
def get_seoul_covid19_100(page_no):
    start_no = (page_no - 1) * 100
    url = f"https://news.seoul.go.kr/api/27/getCorona19Status/get_status_ajax.php?draw={page_no}"
    url = f"{url}&start={start_no}&length=100"
    response = requests.get(url)
    data_json = response.json()
    return data_json

In [555]:
# data available
get_seoul_covid19_100(1)

{'draw': 1,
 'recordsTotal': 41720,
 'recordsFiltered': 41720,
 'data': [["<p class='corona19_no'>71720</p>",
   '225438',
   '2021-08-15',
   '기타',
   '-',
   '감염경로 조사중',
   "<b class=''>-</b>"],
  ["<p class='corona19_no'>71719</p>",
   '225111',
   '2021-08-15',
   '성북구',
   '-',
   '기타 확진자 접촉',
   "<b class=''>-</b>"],
  ["<p class='corona19_no'>71718</p>",
   '225046',
   '2021-08-15',
   '서대문구',
   '-',
   '타시도 확진자 접촉',
   "<b class=''>-</b>"],
  ["<p class='corona19_no'>71717</p>",
   '225288',
   '2021-08-15',
   '강남구',
   '-',
   '감염경로 조사중',
   "<b class=''>-</b>"],
  ["<p class='corona19_no'>71716</p>",
   '224003',
   '2021-08-15',
   '동대문구',
   '-',
   '감염경로 조사중',
   "<b class=''>-</b>"],
  ["<p class='corona19_no'>71715</p>",
   '224959',
   '2021-08-15',
   '강서구',
   '-',
   '감염경로 조사중',
   "<b class=''>-</b>"],
  ["<p class='corona19_no'>71714</p>",
   '225117',
   '2021-08-15',
   '성북구',
   '-',
   '기타 확진자 접촉',
   "<b class=''>-</b>"],
  ["<p class='corona19_no'>71713</p

In [556]:
# data unavailable
get_seoul_covid19_100(-1)

{'draw': -1, 'recordsTotal': 41720, 'recordsFiltered': 41720, 'data': []}

## Python code for combining all pages

In [557]:
# to prevent network demage
import time
# tqdm: process check
from tqdm import trange

In [558]:
def get_multi_page_list(start_page, end_page):
    page_list = []
    for page_no in trange(start_page, end_page + 1):
        one_page = get_seoul_covid19_100(page_no)
        # break when page is empty
        if len(one_page['data']) > 0:
            one_page = pd.DataFrame(one_page['data'])
            page_list.append(one_page)
        else:
            return page_list
    return page_list
        
    


In [559]:
end_page

418

In [560]:
# Test Code
start_page = 1
end_page = round(records_total/ 100) + 1
# end_page = 205
page_list = get_multi_page_list(start_page, end_page)
page_list[:1]

100%|██████████| 418/418 [09:26<00:00,  1.35s/it]


[                                   0       1           2     3  4  \
 0   <p class='corona19_no'>71720</p>  225438  2021-08-15    기타  -   
 1   <p class='corona19_no'>71719</p>  225111  2021-08-15   성북구  -   
 2   <p class='corona19_no'>71718</p>  225046  2021-08-15  서대문구  -   
 3   <p class='corona19_no'>71717</p>  225288  2021-08-15   강남구  -   
 4   <p class='corona19_no'>71716</p>  224003  2021-08-15  동대문구  -   
 ..                               ...     ...         ...   ... ..   
 95  <p class='corona19_no'>71625</p>  224877  2021-08-15   성동구  -   
 96  <p class='corona19_no'>71624</p>  224582  2021-08-15   타시도  -   
 97  <p class='corona19_no'>71623</p>  224870  2021-08-15   마포구  -   
 98  <p class='corona19_no'>71622</p>  224926  2021-08-15   타시도  -   
 99  <p class='corona19_no'>71621</p>  224854  2021-08-15   마포구  -   
 
                         5                  6  
 0                감염경로 조사중  <b class=''>-</b>  
 1               기타 확진자 접촉  <b class=''>-</b>  
 2            

In [561]:
df_all = pd.concat(page_list)
df_all.shape

(41720, 7)

In [562]:
cols = ['Number','Patient','Confirmed Date','Area','Travel History','Transmission Route','Status']

In [563]:
df_all.columns = cols

In [564]:
df_all.head()

,Number,Patient,Confirmed Date,Area,Travel History,Transmission Route,Status
0,<p class='corona19_no'>71720</p>,225438,2021-08-15,기타,-,감염경로 조사중,<b class=''>-</b>
1,<p class='corona19_no'>71719</p>,225111,2021-08-15,성북구,-,기타 확진자 접촉,<b class=''>-</b>
2,<p class='corona19_no'>71718</p>,225046,2021-08-15,서대문구,-,타시도 확진자 접촉,<b class=''>-</b>
3,<p class='corona19_no'>71717</p>,225288,2021-08-15,강남구,-,감염경로 조사중,<b class=''>-</b>
4,<p class='corona19_no'>71716</p>,224003,2021-08-15,동대문구,-,감염경로 조사중,<b class=''>-</b>


In [565]:
df_all.shape

(41720, 7)

## Data Preprocessing

### Number and Status

In [566]:
import re
def extract_number(num_string):
    if type(num_string) == str:
        num_string = num_string.replace("corona19", "")
        num = re.sub("[^0-9]", "", num_string)
        num = int(num)
        return num
    else:
        return num_string

In [567]:
num_string = "<p class='corona19_no'>7625</p>"
extract_number(num_string)

7625

In [568]:
df_all['Number'] = df_all["Number"].map(extract_number)

In [569]:
df_all.head()

,Number,Patient,Confirmed Date,Area,Travel History,Transmission Route,Status
0,71720,225438,2021-08-15,기타,-,감염경로 조사중,<b class=''>-</b>
1,71719,225111,2021-08-15,성북구,-,기타 확진자 접촉,<b class=''>-</b>
2,71718,225046,2021-08-15,서대문구,-,타시도 확진자 접촉,<b class=''>-</b>
3,71717,225288,2021-08-15,강남구,-,감염경로 조사중,<b class=''>-</b>
4,71716,224003,2021-08-15,동대문구,-,감염경로 조사중,<b class=''>-</b>


In [570]:
def extract_hangeul(origin_text):
    subtract_text = re.sub("[^가-힣]", "", origin_text)
    return subtract_text

In [571]:
extract_hangeul("<b class='status1'>퇴원</b>")

'퇴원'

In [572]:
extract_hangeul("<b class='status2'>사망</b>")

'사망'

In [573]:
extract_hangeul("<b class=''>-</b>")

''

In [574]:
df_all["Status"].value_counts()

<b class='status1'>퇴원</b>    32131
<b class='status1'>-</b>      4915
<b class=''>-</b>             4442
<b class='status2'>사망</b>      117
<b class='status2'>-</b>       115
Name: Status, dtype: int64

In [575]:
df_all.loc[df_all["Status"].str.contains("퇴원"), "Status"] = "Recovered"
df_all.loc[df_all["Status"].str.contains("사망"), "Status"] = "Death"
df_all.loc[~df_all["Status"].str.contains("Recovered|Death"), "Status"] = "Quarantined"
df_all

,Number,Patient,Confirmed Date,Area,Travel History,Transmission Route,Status
0,71720,225438,2021-08-15,기타,-,감염경로 조사중,Quarantined
1,71719,225111,2021-08-15,성북구,-,기타 확진자 접촉,Quarantined
2,71718,225046,2021-08-15,서대문구,-,타시도 확진자 접촉,Quarantined
3,71717,225288,2021-08-15,강남구,-,감염경로 조사중,Quarantined
4,71716,224003,2021-08-15,동대문구,-,감염경로 조사중,Quarantined
...,...,...,...,...,...,...,...
15,30005,95804,2021-03-14,송파구,-,타시도 확진자 접촉,Recovered
16,30004,95889,2021-03-14,종로구,-,감염경로 조사중,Recovered
17,30003,95799,2021-03-14,중구,-,중구 소재 주점/음식점 관련,Recovered
18,30002,95872,2021-03-14,종로구,-,기타 확진자 접촉,Recovered


In [576]:
df_all.loc[df_all["Transmission Route"].str.contains("조사중"), "Transmission Route"] = "Under Inspection"
df_all.loc[df_all["Transmission Route"].str.contains("접촉|관련"), "Transmission Route"] = "Known"
df_all.loc[df_all["Transmission Route"].str.contains("해외"), "Transmission Route"] = "Overseas"
df_all.loc[~df_all["Transmission Route"].str.contains("Under Inspection|Known|Overseas"), "Transmission Route"] = "Unknown"
df_all

,Number,Patient,Confirmed Date,Area,Travel History,Transmission Route,Status
0,71720,225438,2021-08-15,기타,-,Under Inspection,Quarantined
1,71719,225111,2021-08-15,성북구,-,Known,Quarantined
2,71718,225046,2021-08-15,서대문구,-,Known,Quarantined
3,71717,225288,2021-08-15,강남구,-,Under Inspection,Quarantined
4,71716,224003,2021-08-15,동대문구,-,Under Inspection,Quarantined
...,...,...,...,...,...,...,...
15,30005,95804,2021-03-14,송파구,-,Known,Recovered
16,30004,95889,2021-03-14,종로구,-,Under Inspection,Recovered
17,30003,95799,2021-03-14,중구,-,Known,Recovered
18,30002,95872,2021-03-14,종로구,-,Known,Recovered


In [580]:
df_all.loc[df_all["Area"].str.contains("종로구"), "Area"] = "Jongno-gu"
df_all.loc[df_all["Area"].str.contains("중구"), "Area"] = "Jung-gu"
df_all.loc[df_all["Area"].str.contains("용산구"), "Area"] = "Yongsan-gu"
df_all.loc[df_all["Area"].str.contains("성동구"), "Area"] = "Seongdong-gu"
df_all.loc[df_all["Area"].str.contains("광진구"), "Area"] = "Gwangjin-gu"
df_all.loc[df_all["Area"].str.contains("동대문구"), "Area"] = "Dongdaemun-gu"
df_all.loc[df_all["Area"].str.contains("중랑구"), "Area"] = "Jungnang-gu"
df_all.loc[df_all["Area"].str.contains("성북구"), "Area"] = "Seongbuk-gu"
df_all.loc[df_all["Area"].str.contains("강북구"), "Area"] = "Gangbuk-gu"
df_all.loc[df_all["Area"].str.contains("도봉구"), "Area"] = "Dobong-gu"
df_all.loc[df_all["Area"].str.contains("노원구"), "Area"] = "Nowon-gu"
df_all.loc[df_all["Area"].str.contains("은평구"), "Area"] = "Eunpyeong-gu"
df_all.loc[df_all["Area"].str.contains("서대문구"), "Area"] = "Seodaemun-gu"
df_all.loc[df_all["Area"].str.contains("마포구"), "Area"] = "Mapo-gu"
df_all.loc[df_all["Area"].str.contains("양천구"), "Area"] = "Yangcheon-gu"
df_all.loc[df_all["Area"].str.contains("강서구"), "Area"] = "Gangseo-gu"
df_all.loc[df_all["Area"].str.contains("구로구"), "Area"] = "Guro-gu"
df_all.loc[df_all["Area"].str.contains("금천구"), "Area"] = "Geumcheon-gu"
df_all.loc[df_all["Area"].str.contains("영등포구"), "Area"] = "Yeongdeungpo-gu"
df_all.loc[df_all["Area"].str.contains("동작구"), "Area"] = "Dongjak-gu"
df_all.loc[df_all["Area"].str.contains("관악구"), "Area"] = "Gwanak-gu"
df_all.loc[df_all["Area"].str.contains("서초구"), "Area"] = "Seocho-gu"
df_all.loc[df_all["Area"].str.contains("강남구"), "Area"] = "Gangnam-gu"
df_all.loc[df_all["Area"].str.contains("송파구"), "Area"] = "Songpa-gu"
df_all.loc[df_all["Area"].str.contains("강동구"), "Area"] = "Gangdong-gu"
df_all.loc[df_all["Area"].str.contains("타시도|기타"), "Area"] = "Another City"

In [581]:
file_name = 'covid_19_recent_data.csv'
df_all.to_csv(file_name, index=False)

In [582]:
pd.read_csv(file_name)

,Number,Patient,Confirmed Date,Area,Travel History,Transmission Route,Status
0,71720,225438,2021-08-15,Another City,-,Under Inspection,Quarantined
1,71719,225111,2021-08-15,Seongbuk-gu,-,Known,Quarantined
2,71718,225046,2021-08-15,Seodaemun-gu,-,Known,Quarantined
3,71717,225288,2021-08-15,Gangnam-gu,-,Under Inspection,Quarantined
4,71716,224003,2021-08-15,Dongdaemun-gu,-,Under Inspection,Quarantined
...,...,...,...,...,...,...,...
41715,30005,95804,2021-03-14,Songpa-gu,-,Known,Recovered
41716,30004,95889,2021-03-14,Jongno-gu,-,Under Inspection,Recovered
41717,30003,95799,2021-03-14,Jung-gu,-,Known,Recovered
41718,30002,95872,2021-03-14,Jongno-gu,-,Known,Recovered
